# Testing

We perform testing to increase confidence that our programs work correctly and that those programs meet our customer's expectations. We call this type of testing functional testing as it ensures that the code functions correctly. While it sounds trite, one cannot overstate the importance of software testing. Software and computer systems permeate our daily lives; the correct functionality of those systems is paramount to our safety and well-being. Unless you are reading these words from a dead tree off the grid somewhere, you are either working with a computer system or benefiting from a product (electricity) controlled by a computer system. Not only can software defects waste time as we work through the issues, but those defects can lead to monetary loss (both direct and indirect) and possibly death.  

Testing directly improves product quality, leading to higher customer satisfaction with systems. For example, imagine using a computer that crashes regularly. How long would you continue to use that system? Testing also decreases product development costs, as finding and fixing defects sooner to when they first appear in the system costs less money to fix.

Yes, you can initially get away with manually testing your software as you write code. (In some ways, this is necessary - build a little, test that those parts work, and build some more. Rinse, Lather, Repeat.). However, eventually, you will have significant maintainability issues. As you make changes, how do you ensure those changes do not break something?

Before the early 2000s, most software and system testing was a manual process. Individuals wrote test scripts that others manually followed. Some developers built unit test cases but then executed those test cases manually. Since then, testing has become predominantly automated (although not universally). 

Several advantages to automated testing:
1. Cost-savings.  These test cases can be run repetitively. Yes, higher upfront costs exist to develop test cases, but that is a one-time expense. Over the lifetime of a project, test cases can be executed thousands of times.
2. Faster development timeframes.  As developers make enhancements to a system, existing test cases can be executed to ensure the existing functionality works. (regression testing)
3. Immediate feedback
4. Automation of test case development.  Fuzzing - https://owasp.org/www-community/Fuzzing
5. Automated testing is foundational to continuous integration, continuous delivery, and other modern [DevOps practices](https://about.gitlab.com/topics/devops/).
6. Oh ...  by the way, the automated grading of your program submissions - that's all unit tests.

This notebook focuses on verification of the code we have been developing - primarily this
involves unit testing of the code we write. Generally speaking, unit tests check that the software produces the correct output based on a specific input. We compare actual and expected results to determine if an error occurred. We consider a "unit" to be any portion of code testable in isolation. We can call functions and execute them, so functions are units. However, we cannot call specific lines within a function, so they are smaller than a unit. As we examine classes shortly, we can test many of the class components in isolation and, thus, treat classes as units for testing.

The primary structure of functional tests is relatively consistent. As these tests check that the system produces the correct output for a given input, functions tests following this outline:
1. Define inputs
2. Identify the expected output
3. Perform any system preparation to execute the test
4. Execute the test
5. Get the actual output
6. Compare the actual output against the expected output to see if the two match.

This notebook walks through developing test cases to execute on an automated basis. Our overarching goal is two-fold:
1. Deliver high-quality products to our customers
2. Reduce costs as much as possible.

We will use Python's built-in testing framework, `unittest`. This framework provides capabilities to organize test cases, execute code, establish any pre-conditions necessary to perform tests, execute the tests, and remove any artifacts of the testing process. The test cases work by making assertions about the code - does this result match some expected value? An assertion is a statement of fact<sup>[*](https://www.lexico.com/en/definition/assertion)</sup>. As such, if an assertion fails, our assumption is incorrect, or the code produces an incorrect value.

[unittest documentation](https://docs.python.org/3/library/unittest.html)

## Case Study: Bond Valuation
We will implement several functions to compute various bond yield values to produce some code to test.

A bond represents a type of corporate debt. A corporation issues a bond with a fixed principal that will be paid to the bond owner at maturity and fixed interest payments at set times. For instance, a corporation can issue a $\$$1,000 bond with 10$\%$ annual interest with a maturity of 10 years. Investors holding on to this bond receive $\$$2,000 over the lifetime (10 payments of $\$$100 plus $\$1,000 at the end).

One valuation to examine is the bond yield.  $bond yield = \frac{annual~coupon~payment}{bond~price} $  where the $annual~coupon~payment = face~value * coupon~rate$

We can then define a function to implement the bond yield.

In [ ]:
def compute_bond_yield(bond_price, face_value, coupon_rate):
    annual_coupon_payment = face_value * coupon_rate
    bond_yield = annual_coupon_payment / face_value         # used face_value instead of bond price
    return bond_yield

In [ ]:
compute_bond_yield(970,1000,0.05)

The result looks correct, but is it really? Ideally, we want to ensure that the output matches the expected. However, if we plug those numbers into a bond yield calculator, we see that the result should be 0.0515.

To help find issues like this, we need to write unit tests. We can write these tests firsts and then build the function. We know our functions then work when they pass the test cases.

To use the `unittest` module, we will first need to import that module and then write test cases. As with exceptions, we will need to extend a pre-existing class and then add methods to that class.  The approach will follow this outline:

<code>
    
import unittest

class Test<i>Name</i>(unittest.TestCase):
    def setUp(self):
        pass

    def tearDown(self):
        pass

    def test_<i>name_1</i>(self):
        ...
    
    def test_<i>name_2</i>(self):
        ...
    
    def test_<i>name_n</i>(self):
        ...
    
</code>  

Generally, you will want to identify code that contains test cases unmistakably. As such, it makes sense to start the class names for the unit test with "Test".  Similarly, we will also want to give descriptive names to the individual methods. Many testing frameworks assume that methods starting with `test_` are test cases.  We can also add docstrings to the methods to provide a more detailed description of the test.

The framework calls the `setup()` method before each test method. Use this capability to allocate resources or set up conditions necessary to execute the test cases.  You can also define a `setupClass()` method that runs once before executing any tests in the class ([more details](https://docs.python.org/3/library/unittest.html#unittest.TestCase.setUpClass)). 

The `tearDown()` method is called after each test method, regardless of any exceptions.  Use this method to release any allocated resources or move the system state back before the test case executed. `tearDownClass()` also exists - this executes after all of the tests in the class have been executed.

It was not necessary to define `setup()` and `tearDown()` in these examples - the default behavior for both does nothing.


From within a Python program, we can execute the defined test cases with the following method call:
<code>
    
    unittest.main(argv=['unittest','Test<i>Name</i>'], verbosity=2, exit=False)

</code>

Remove the 'Test</i>Name</i>' element to execute all test cases that have been loaded by the interpreter.

From the command line, we can execute
`python -m unittest TestName`

To discover all possible tests cases and run those, use 
`python -m unittest discover`

Following the test outline, we will write some test cases for the `compute_bond_yield()` function.

In [ ]:
class TestBondYield(unittest.TestCase):
    "Validates compute_bond_yield"
    def setUp(self):
        pass
    def tearDown(self):
        pass
    
    def test_bond_yield_5_percent(self):
        "Validate that the computed bond yield approximates 0.0515"
        bond_yield = compute_bond_yield(970, 1000, 0.05)
        self.assertAlmostEqual(bond_yield, 0.0515, places=4)
        
    def test_bond_yield_0_percent(self):
        "Validate that the computed bond yield equals 0 for 0%"
        bond_yield = compute_bond_yield(970, 1000, 0.00)
        self.assertEqual(bond_yield, 0.0)        

In [ ]:
unittest.main(argv=['unittest','TestBondYield'], verbosity=2, exit=False)

One of the tests passed, while the other test failed. We now need to debug the code to see what occurred. Since this is a relatively small amount of code, we can employ the debug strategy of "read" and then "run". As we read the source code, does it match up with the formula?  No.  We can also step through the code manually (either with a debugger or using a memory diagram/tracing approach).  

Let's correct the code and then re-run the test.

In [ ]:
def compute_bond_yield(bond_price, face_value, coupon_rate):
    annual_coupon_payment = face_value * coupon_rate
    bond_yield = annual_coupon_payment / bond_price
    return bond_yield

In [ ]:
unittest.main(argv=['unittest','TestBondYield'], verbosity=2, exit=False)

Another valuation for bonds is the yield to maturity(YTM).  This value is the speculative rate of a return given that an investor purchases a bond a given current market price and then holds the bond until marturity (thus, all interest payments and final payments are made).

The YTM can be approximated with this formula: 

$ YTM = \frac{C + \frac{FV-PV}{t}}{\frac{FV+PV}{2}} $
where 
- $C$ – Interest/coupon payment
- $FV$ – Face value of the security
- $PV$ – Present value/price of the security
- $t$ – How many years it takes the security to reach maturity

In [ ]:
def calculate_yield_to_maturity(bond_price, face_value, coupon_rate, t):
    c = coupon_rate * face_value
    ytm = (c + (face_value - bond_price)/t) / ( (face_value+bond_price) /2 )
    return ytm

In [ ]:
print(compute_bond_yield(850,1000,0.15))
print(calculate_yield_to_maturity(850,1000,0.15,7))

We can then develop another set of test cases to test that function:

In [ ]:
class TestBondYTM(unittest.TestCase):
    def setUp(self):
        pass
    def tearDown(self):
        pass
    
    def test_bond_ytm_5_per_10_year(self):
        "Validate that the computed bond yield approximates 0.0515"
        bond_yield = calculate_yield_to_maturity(970, 1000, 0.05,10)
        self.assertAlmostEqual(bond_yield, 0.0538, places=4)
  

In [ ]:
unittest.main(argv=['unittest','TestBondYTM'], verbosity=2, exit=False)

## Documenting Test Cases
To document a test case, we generally define this information:
- Unique Identifier
- Test Inputs
- Expected Results
- Actual Results

As you look at the source code, you can see that the `unittest` classes have this information in them already. You should include an identifier in the docstring - this will help you locate test cases in larger projects and possibly provide a hierarchical organization if desired.

If you do manually test code instead, you should keep track of your tests in a text file. That way, you at least have some reference to how you previously tested the code. However, the *right* way to do this is to create automated unit tests. The investment will be worth it.

## Black Box and White Box Testing

As we approach functional testing, we can develop test cases based on two approaches: black box and white box. In the black box approach, we do not have any access to the underlying source code - we need to consider the test cases solely based upon the inputs and the corresponding outputs. In contrast, the white box approach assumes that the test developers can see the code and then develop code specific to test paths, conditions, and branches that the code can follow. For unit testing, developers will combine these approaches to build test cases. For user acceptance testing, test cases follow a black-box approach as the users/testers do not have insight into the underlying pinnings as to how a system works.

Revisiting the definition of software testing from the [Software Engineering Body of Knowledge](http://www.swebok.org):
> Software testing consists of the dynamic verification that a program provides expected behaviors on a finite set of test cases, suitably selected from the usually infinite execution domain.

The definition presents several interesting ideas:  
1. The input set quite often can be infinite.  Even for the simple examples above, we have endless input values - use different values in the decimal places for the parameters to the functions.  
2. We need to choose (create) test cases that are "suitably selected".  In other words, what test cases are most likely to find potential issues, represent the range of input values, and execute all of the code.

### Black Box Testing: Equivalence Classes
One strategy for black box testing is to divide the input values into different equivalence classes.

For example, U.S. zip codes are five digits long and must conatin only numbers.. So we can divide this into the following classes:
- a string less than five characters in length (invalid)
- a string more than five characters in length (invalid)
- a string that is five characters long but has non-numeric characters present  (invalid)
- a string that is five characters long with only digits 0-9 present. (valid)

Using these four classes, we assume inputs within each class are equivalent to each other. E.g., testing "" and "123" will be the same.  Similarly, no value exists for testing the 10,000 valid possibilities beyond just one example.

As we use equivalence classes to create test cases, we 
1. Determine if there are any specified limits and conditions for input values.
2. Break in the input space into different classes to create partitions:
   1. If the input space is specified as a range condition, then three classes are created: below the range (invalid), in the range (valid), and above the range (invalid).  For example, if the condition is an individual's age must be between 18 and 65, the classes are below 18, 18 to 65, and above 65.
   2. If the input space is a given input, we have at least two classes:  valid input and invalid inputs.  Depending upon the type (e.g., numbers), we may have more classes for the invalid inputs.
   3. C.	If we look at membership within a collection, we have two equivalence classes: the member exists or the member does not exist.
   4. For Boolean values, we have two classes: `True` and `False`.
3. Write a test case that tests each range's "middle" input values.

Examples:
- We could have a business rule that stock symbols must be between 1 and 5 characters in length.  With these, we would create equivalence classes looking at strings that are 0, 3, and 7 characters in length.
- Based on temperature and pressure, elements and other substances exist in one of three states - solid, liquid, and gas. Therefore, we would define equivalence classes for the input ranges to test each state.
- The following table presents the 2022 U.S. Federal Tax Brackets:
  | Income  | Tax   |
  |---------|:------|
  | <= $\$$10,275 | 10% of the taxable income |
  | > $\$$10,275 and <= $\$$41,775  | $\$$1,027.50 plus 12% of the excess over $\$$10,275 |
  | > $\$$41,775 and <= $\$$89,075  | $\$$4,807.50 plus 22% of the excess over $\$$41,775 |
  | > $\$$89,075 and <= $\$$170,050	| $\$$15,213.50 plus 24% of the excess over $\$$89,075|  
  | > $\$$170,050 and <= $\$$215,950 | $\$$34,647.50 plus 32% of the the excess over $\$$170,050 |
  | > $\$$215,950 and <= $\$$539,900 | $\$$49,335.50 plus 35% of the excess over $\$$215,950 |
  | > $\$$539,900 |	$\$$162,718 plus 37% of the excess over $539,900
  
  We would use seven equivalence classes to create test cases with these tax brackets. We choose a value in the middle of each range to represent that test case.

### Black Box Testing: Boundary Value Analysis
Boundary value analysis (boundary testing) is another black box testing strategy that complements equivalence classes. Programmers often make mistakes at the boundaries of input ranges. For example, they may use just a less than `<` comparison operator rather than a less than equals `<=` comparison operator. As such, we create test cases that are just below the boundary, on the boundary, and just above the boundary. 

For testing the US Federal tax brackets, we would use: 
  $\$$10,274, $\$$10,275, $\$$10,276, $\$$41,774, $\$$41,775, $\$$41,776, $\$$89,074, $\$$89,075, $\$$89,076,
  $\$$170,049, $\$$170,050, $\$$170,051, $\$$215,949, $\$$215,950, $\$$215,951, $\$$539,899, $\$$539,900, and
  $\$$539,901.  

Whew,  no one ever said U.S. Federal Taxes were easy.  As a side note, payroll is a surprisingly challenging problem domain. Systems have to deal with various employee types, salaries, and pay mechanisms, but they  also have to consider benefits (e.g., 401k) and taxes (country, state, local).

Another example would be looking at the states (solid, liquid, and gas). At less than 0 degrees Celsius, water becomes solid. Between 0 degrees Celsius and 100 degrees Celsius, water exists as a liquid. Above 100 degrees Celsius, water becomes a gas. So now, we have -1, 0, 1, 99, 100, and 101 as boundary possibilities.


As numbers go from negative to positive (-1, 0, -1) is another good location for boundary testing.

For collections, we would look at if the collection was empty or a single member.

For lists, look at the starting and ending indexes.
  


### White Box Testing: Coverage 
With white box testing, developers examine the source to create test cases. The immediate advantage is the ability to look at the different conditionals within the code and write specific test cases against those conditionals. More importantly, the primary consideration with white box testing is test coverage - having a set of test cases that execute all possible code. To determine how much of the code does execute, we look at several different levels of test coverage:
- statement
- decision (branch)
- paths

To help define and understand these coverage concepts, consider the following function `calculate_price()`:

In [ ]:
def calculate_price(stock_price, num_items, discount):
    quantity_discount = 0.0
    result = 0.0

    if num_items >= 5:
        quantity_discount = 10;
    elif num_items >= 10:          #ordering mistake deliberately placed in code
        quantity_discount = 15;
    else:
        quantity_discount = 0

    if discount > quantity_discount:
        quantity_discount = discount

    result = stock_price/100.0 * (100-quantity_discountquant) * num_items;

    return result;

From this code, we can create a [control-flow graph](https://en.wikipedia.org/wiki/Control-flow_graph), which is a graphical representation of the paths that execution may take through a program. Each node represents a block - a piece of code - that executes as a single unit. The edges represent decisions (or jumps with iteration) in the control flow. The primary difference between a control-flow graph and a flowchart is that the decision node is combined with the predecessor node (if there is only one) as both execute as a unit. 

![](https://github.com/slankas/DataScienceNotebooks/blob/main/IntroductionToPython/images/calculate_price_label.png?raw=true)

For full statement coverage, each node must execute. The function does not use the parameter `stock_price` in any conditionals, so we can keep that value constant. We need to use three different values (0, 5, 10) of `num_items` to execute the first set of conditionals. Then we need at least one of the test cases (or a new one) where the discount value is higher than the computed quantity discount. When we test `num_items` =10, we will set `discount` = 20.

However, after walking through the test case of num_items = 10, we realize that the conditional in node "2" never evaluates to `True`. After executing the three test cases, we can see through coverage analysis that node "4" never executes.  Hence, we can see that a logic error exists.  The statement coverage metric is 87.5% (7/8).
- For `num_items` = 0 and `discount` = 0, we execute nodes 1, 2, 5, 6, 8
- For `num_items` = 5 and `discount` = 0, we execute nodes 1, 3, 6, 8
- For `num_items` =10 and `discount` = 20, we execute nodes 1, 2, 5, 6, 7, 8

Decision coverage requires executing all of the possible outcomes of decisions within the program. Using the control-flow graph, this requires traversing each edge in the graph. Using the same set of test cases:
- For `num_items` = 0 and `discount` = 0, we follow edges B, D, G, I
- For `num_items` = 5 and `discount` = 0, we follow edges A, E, I
- For `num_items` =10 and `discount` = 20, we follow edges B, D, G, H, J

As before, no possibility exists to follow edges C and F. The branch coverage metric is 80% (8/10).

Path coverage requires the test cases to follow all possible paths through the control flow graph.  
For paths, we have six possibilities.  
- A E H J
- A E I
- B C F H J
- B C F I
- B D G H J
- B D G I

One way to compute the total number of paths is to examine the "independent" conditionals within the program.  In this example, our first conditional had three possibilities, while the second conditional had two possibilities. $ 2 * 3 = 6 $ 

The number of paths can grow to become very large. For example, the U.S. Federal Tax Bracket has seven initial possibilities assuming we only looked at just the income. Four independent `if` statements within a function creates $2^4(16)$ possibilities. As you can see, getting to 100% path coverage can become overwhelming with nontrivial code.

Also, realize that these coverage metrics can be misleading. One test case alone (`num_items` =10 and `discount` = 20) executes 6 of 8 nodes (statements) and 5 of 10 edges (decisions/branches). Assuming, we corrected the code to order the conditionals correctly in the first check:
<pre>
    if num_items >= 10:
        quantity_discount = 15;
    elif num_items >= 5:       
        quantity_discount = 10;
    else:
        quantity_discount = 0    
</pre>
Our three test cases would then have 100% statement and 100% decision coverage. Assuming we add another three test cases to get to 100% path coverage, it may still be possible to have issues with the code.  What happens when the `discount` is greater than or equal to 100?  Are we giving away products as well as handing out money?
    
Additionally, coverage tools only evaluate whether or not code executes - they do not evaluate whether a test case is beneficial. As a result, you can have test cases that execute quite a bit of code but do not perform worthwhile tests or have meaningful assertions. Combining both black box test strategies and white box strategies helps to overcome such gaps in test cases.

## Testing Strategies

Many different strategies exist for developing test cases. Paul Gries, Jennifer Campbell, and Jason Montojo presented these strategies:

1. Think about size. When a test involves a collection such as a list, string, dictionary, or file, you need to do the following:
  1. Test the empty collection.
  2. Test a collection with one item in it.
  3. Test a general case with several items.
  4. Test the smallest interesting case, such as sorting a list containing two values.

2. Think about dichotomies. A dichotomy is a contrast between two things. Examples of dichotomies are empty/full, even/odd, positive/negative, and alphabetic/nonalphabetic. If a function deals with two or more different categories or situations, make sure you test all of them.
3. Think about boundaries. If a function behaves differently around a particular boundary or threshold, test exactly that boundary case.
4. Think about order. If a function behaves differently when values appear in different orders, identify those orders and test each one of them. For the sorting example mentioned earlier, you’ll want one test case where the items are in order and one where they are not.

Source: Paul Gries, Jennifer Campbell, and Jason Montojo. 2017. _Practical Programming: An Introduction to Computer Science Using Python 3.6 (3rd. ed.)_. Pragmatic Bookshelf.

Andrew Hunt and David Thomas created an acronym: CORRECT
- Conformance: Does the value conform to an expected format?
- Ordering: Is the set of values ordered or unordered as appropriate?
- Range: Is the value within reasonable minimum and maximum values?
- Reference: Does the code reference anything external that is not under the direct control of the code itself?
- Existence: Does the value exist (is it non-null, nonzero, present in a set, and so on)?
- Cardinality: Are there exactly enough values?
- Time (absolute and relative): Is everything happening in order? At the right time? In time?

Source: Andrew Hunt and David Thomas. 2003. _Pragmatic Unit Testing in Java with JUnit_. The Pragmatic Programmers.

Some additional guidelines to consider:
- Choose inputs to force the system to generate all error messages
- Choose inputs to force the system to raise exceptions
- Test both the "happy" paths as well as as the negative times
- Execute APIs with different orderings
- Look for inputs that may cause overflow issues.  Use extremely large or small values.

<div style="border: 3px solid black;padding: 10px; border-radius: 10px;">
    <b>Testing Miss:</b>
One of the testing misses that one of the authors witnessed looked at checking the 401k deductions that 
individual could make.  By law, this value is limited to $\$$27,000 for individuals over the age of 50.  While running the code in production, the process failed as one employee (the company CEO) reached the limit on the first paycheck of year.
</div>

## How Much Testing?
Writing a comprehensive test suite is a time-consuming effort. Unfortunately, how much testing is required is project dependent. Much depends upon the confidence needed that the code is correct. Little testing may be necessary for throw-away code that may be used once or to evaluate a particular design choice. For code used in mission(business) critical systems, much more confidence is needed that the code is correct; therefore, you need to build more test cases. Ideally, you should aim for 100% decision coverage. Testing is an investment that will save you time and resources. You should be able to find issues sooner and confidently make future code changes without breaking existing functionality.

Very little exists in the literature (academic papers and books) for guidelines.  Fred Books, in his seminal book, _The Mythical Man-Month: Essays on Software Engineering_, presented this schedule breakdown:
- 1/3 design
- 1/6 coding
- 1/2 testing (broken down evenly between unit testing and system testing)

Source: F. P. Brooks, Jr., _The Mythical Man-Month: Essays on Software Engineering_, Anniversary Edition, Boston: Addison-Wesley, 1995.

Final thoughts:
- Test early: start testing as soon as parts are implemented
- Test often: running tests at every reasonable opportunity

## Other Testing Frameworks

`unittest` is not the only test framework available for Python. Python's standard library contains `unittest`.  The framework also matches up similarly to the widely-used JUnit framework for Java.

Other frameworks to examine: 
- Pytest: https://docs.pytest.org/
- Hypothesis: https://hypothesis.readthedocs.io/

## Exercises
1. For the following problem, determine the equivalence classes and provide a representative value:  For orders below $\$$100,000 no discount is available.  For orders up to $\$$200,000, a discount of 10%.  For orders up to $\$$350,000, a discount of 15% is available.  For orders over $\$$350,000, a 20% is offered.  
1. For the values in the previous problem, what order values are needed to test the boundary conditions?
1. What are the boundary values to test for the individual's 's age must be between 18 and 65?
1. Write the function `compute_price(order_total)` and create a `unittest` class for both the boundary conditions and equivalence classes.
1. Write a function `strip_digits(str)` that removes any digits from the string parameter and then returns the result. Write a `unittest` class for the function.  You should test the following conditions:
   - empty string
   - string with no digits present
   - string with only digits present
   - string with a mixture of digits and other characters.
   
1. Given the following function, `median(l)`, which returns the median value for the items in a list, create a `unittest` class that has 100% statement coverage and 100% decision coverage.  How many paths are possible?

In [ ]:
def median(l):
    """Finds the median value of the list"""
    if l:
        result = 0
        s_list = sorted(l)
        if len(s_list) % 2 == 1:
            result = s_list[len(s_list)//2] 
        else:
            print(len(s_list)//2)
            result = (s_list[len(s_list)//2 - 1] + s_list[len(s_list)//2])/2
        return result
    else:
        raise ValueError("list empty")

![](https://github.com/slankas/DataScienceNotebooks/blob/main/IntroductionToPython/images/median.png?raw=true)

<pre>
